In [ ]:
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import xarray as xr
import pandas as pd
import numpy as np

from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.ticker import AutoMinorLocator
from cartopy.util import add_cyclic_point
from matplotlib import rcParams
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from cartopy.util import add_cyclic_point
from scipy.stats import linregress

import cftime
from datetime import datetime

import regionmask

## Historical Data

In [ ]:
data1 = '/Users/...UKESM1_historical_1981-2010.cvdp_data.1850-2014.nc'

In [ ]:
hist = xr.open_dataset(data1)
hist

In [ ]:
ta_vars = [var for var in hist.variables if 'albedo' in var]
ta_vars

In [ ]:
a = 'sam_ann'
sam_ann = hist['sam_ann']
sam_ann.plot()

## SSP Data for When Comparing to Historical 

In [ ]:
# ssp126
data2 = '/Users/...SSP126/UKESM1_ssp126_1981-2010.cvdp_data.1850-2100.nc'

In [ ]:
ssp126hist = xr.open_dataset(data2)
ssp126hist

In [ ]:
# ssp585
data3 = '/Users/...SSP585/UKESM1_ssp585_1981-2010.cvdp_data.1850-2100.nc'

In [ ]:
ssp585hist = xr.open_dataset(data3)
ssp585hist

## SSP Data for Mean Averages (2071-2100)

In [ ]:
# ssp126
data4 = '/Users/...SSP126/UKESM1_ssp126_2071-2100.cvdp_data.1850-2100.nc'

In [ ]:
ssp126mean = xr.open_dataset(data4)
ssp126mean

In [ ]:
# ssp585
data5 = '/Users/...SSP585/UKESM1_ssp585_2071-2100.cvdp_data.1850-2100.nc'

In [ ]:
ssp585mean = xr.open_dataset(data5)
ssp585mean

In [ ]:
a = 'sam_ann'
sam_ann = ssp126mean['sam_ann']
sam_ann.plot()

In [ ]:
a = 'sam_ann'
sam_ann = ssp585mean['sam_ann']
sam_ann.plot()

# Figure 1: Plotting SST and SIE, SAM and SIE - Historical Timeseries

In [ ]:
# SIE
sic_var1 = 'sic_sh_extent_ann'
sie_hist = hist[sic_var1]
sie_hist

In [ ]:
# TAS
sst_var1 = 'tas_global_avg_ann'
sst_hist = hist[sst_var1]
sst_hist

In [ ]:
# SAM
sam_var1 = 'sam_timeseries_ann'
sam_hist = hist[sam_var1]
sam_hist

In [ ]:
# calc smoothed data
sie_smoothed = pd.Series(sie_hist).rolling(window=5, min_periods=1).mean()
sst_smoothed = pd.Series(sst_hist).rolling(window=5, min_periods=1).mean()
sam_smoothed = pd.Series(sam_hist).rolling(window=5, min_periods=1).mean()

# calc std from raw vals
sie_std_raw = pd.Series(sie_hist).rolling(window=5, min_periods=1).std()
sst_std_raw = pd.Series(sst_hist).rolling(window=5, min_periods=1).std()
sam_std_raw = pd.Series(sam_hist).rolling(window=5, min_periods=1).std()

# subplots
fig, axs = plt.subplots(2, 1, figsize=(14, 9), sharex=True)

# SIE and TAS
color1 = 'tab:blue'
axs[0].set_xlabel('Year', fontsize=12)
axs[0].set_ylabel('Sea Ice Extent ($10^{12}$ m$^2$)', fontsize=12)
sie_plot = axs[0].plot(hist['TIME'], sie_smoothed, color=color1, label='Sea Ice Extent')
axs[0].fill_between(hist['TIME'], sie_smoothed - sie_std_raw, sie_smoothed + sie_std_raw, color=color1, alpha=0.2)
axs[0].tick_params(axis='y')
axs[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1f}'))
axs[0].set_title('', fontsize=12)

color2 = 'tab:red'
ax2 = axs[0].twinx()
ax2.set_ylabel('Temperature Above Surface (°C)', fontsize=12)
sst_plot = ax2.plot(hist['TIME'], sst_smoothed, color=color2, label='Temperature Above Surface')
ax2.fill_between(hist['TIME'], sst_smoothed - sst_std_raw, sst_smoothed + sst_std_raw, color=color2, alpha=0.2)
ax2.tick_params(axis='y')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))

# SAM and SIE
color3 = 'mediumorchid'
axs[1].set_xlabel('Year', fontsize=12)
axs[1].set_ylabel('Sea Ice Extent ($10^{12}$ m$^2$)', fontsize=12)
sie_plot_2 = axs[1].plot(hist['TIME'], sie_smoothed, linestyle='-', label='Sea Ice Extent')
axs[1].fill_between(hist['TIME'], sie_smoothed - sie_std_raw, sie_smoothed + sie_std_raw, color=color1, alpha=0.2)
axs[1].tick_params(axis='y')
axs[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1f}'))
axs[1].set_ylim(10, 16)  # y-axis limit for SIE

ax3 = axs[1].twinx()
ax3.set_ylabel('Southern Annular Mode Index', fontsize=12)
sam_plot = ax3.plot(hist['TIME'], sam_smoothed, color=color3, label='Southern Annular Mode')
ax3.fill_between(hist['TIME'], sam_smoothed - sam_std_raw, sam_smoothed + sam_std_raw, color=color3, alpha=0.2)
ax3.tick_params(axis='y')
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))

# line styles, legends
lines1, labels1 = axs[0].get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = axs[1].get_legend_handles_labels()
lines4, labels4 = ax3.get_legend_handles_labels()
legend1 = axs[0].legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=10)
legend2 = axs[1].legend(lines3 + lines4, labels3 + labels4, loc='upper right', fontsize=10)
legend1.set_bbox_to_anchor((0.22, 1))
legend2.set_bbox_to_anchor((0.195, 1))

# gridlines
for ax in axs:
    ax.grid(which='major', linestyle='-', linewidth=0.2, color='black')
    ax.grid(which='minor', linestyle=':', linewidth=0.15, color='black')
    ax.set_xticks(np.arange(hist['TIME'][0], hist['TIME'][-1] + 1, 10))
    ax.set_xticks(np.arange(hist['TIME'][0] + 5, hist['TIME'][-1] + 1, 10), minor=True)
    ax.set_yticks(np.arange(10, 17, 1))
    ax.set_yticks(np.arange(10.5, 16.5, 1), minor=True)

plt.tight_layout()
plt.suptitle('', fontsize=14)
plt.show()

In [ ]:
tas_vars = [var for var in hist.variables if 'tas' in var]
tas_vars

In [ ]:
var_11 = 'tas_global_avg_ann'
tas = hist[var_11]
tas.plot()

In [ ]:
sst_hist.plot()

In [ ]:
varr = 'ipcc_ANT_ocn_tas'
tas_ant_ocn = hist[varr]
tas_ant_ocn.plot()

In [ ]:
# available regions
regionmask.defined_regions.ar6.all.plot(text_kws=dict(color="#67000d", fontsize=7, bbox=dict(pad=0.2, color="w")))
print(regionmask.defined_regions.ar6.all)

In [ ]:
var_111 = 'ipcc_ENA_tas'
tas_ENA = hist[var_111]
tas_ENA

### Calculating Simulation Variation for Historical SSP126 and SSP585:

In [ ]:
# sam hist
sam_hist

In [ ]:
# ssp126
sam_var2 = 'sam_timeseries_ann'
sam_126 = ssp126hist[sam_var2]
sam_126

In [ ]:
# ssp585
sam_var3 = 'sam_timeseries_ann'
sam_585 = ssp585hist[sam_var3]
sam_585

In [ ]:
# merging timeseries into dfs for easier analysis / plotting
df_hist = sam_hist.to_dataframe(name="SAM Historical")
df_126 = sam_126.to_dataframe(name="SAM SSP126")
df_585 = sam_585.to_dataframe(name="SAM SSP585")

# merging with TIME
sam_sim_vari = pd.merge(df_hist, df_126, on="TIME", how="outer")
sam_sim_vari = pd.merge(sam_sim_vari, df_585, on="TIME", how="outer")

# reindexing for time period
sam_sim_vari = sam_sim_vari.reindex(range(1850, 2101))

# replacing NaNs > zero's
sam_sim_vari["SAM Historical"].fillna("0.00", inplace=True)

# column renaming
sam_sim_vari.columns = ["SAM Historical", "SAM SSP126", "SAM SSP585"]

sam_sim_vari.tail()

In [ ]:
# adding sumlation variations

# ssp126
sam_sim_vari["Simulation Variation SSP126"] = sam_sim_vari.loc[1850:2014, "SAM SSP126"] - df_hist.loc[1850:2014, "SAM Historical"]

# ssp585
sam_sim_vari["Simulation Variation SSP585"] = sam_sim_vari.loc[1850:2014, "SAM SSP585"] - df_hist.loc[1850:2014, "SAM Historical"]

sam_sim_vari.head()

In [ ]:
# converting relevant columns to numeric (had problems plotting)
sam_sim_vari["SAM Historical"] = pd.to_numeric(sam_sim_vari["SAM Historical"], errors="coerce")
sam_sim_vari["SAM SSP126"] = pd.to_numeric(sam_sim_vari["SAM SSP126"], errors="coerce")
sam_sim_vari["Simulation Variation SSP126"] = pd.to_numeric(sam_sim_vari["Simulation Variation SSP126"], errors="coerce")

sam_sim_vari.fillna(value={"SAM Historical": 0, "SAM SSP126": 0, "Simulation Variation SSP126": 0}, inplace=True)

# checking for inf values
sam_sim_vari.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
sam_sim_vari["SAM Historical (Rolling)"] = sam_sim_vari["SAM Historical"].rolling(window=5, min_periods=1).mean()

# rolling avgs
sam_sim_vari["Simulation Variation SSP126 (Rolling)"] = sam_sim_vari["Simulation Variation SSP126"].rolling(window=5, min_periods=1).mean()
sam_sim_vari["Simulation Variation SSP585 (Rolling)"] = sam_sim_vari["Simulation Variation SSP585"].rolling(window=5, min_periods=1).mean()

historical_start = 1850
historical_end = 2014

fig, ax = plt.subplots(figsize=(16, 5))

# major / minor gridlines
ax.grid(which='both', color='gray', linestyle='-', linewidth=0.3)
ax.minorticks_on()
ax.grid(which='minor', color='gray', linestyle=':', linewidth=0.1)  # Minor gridlines

# SAM historical
ax.plot(sam_sim_vari.loc[historical_start:historical_end].index, 
        sam_sim_vari.loc[historical_start:historical_end, "SAM Historical (Rolling)"], 
        label="SAM Historical")

# fill between SAM historical (rolling avg) and ssp126 variation
ax.fill_between(sam_sim_vari.loc[historical_start:historical_end].index, 
                sam_sim_vari.loc[historical_start:historical_end, "SAM Historical (Rolling)"] - sam_sim_vari.loc[historical_start:historical_end, "Simulation Variation SSP126 (Rolling)"], 
                sam_sim_vari.loc[historical_start:historical_end, "SAM Historical (Rolling)"] + sam_sim_vari.loc[historical_start:historical_end, "Simulation Variation SSP126 (Rolling)"], 
                color='dimgray', alpha=0.5, label="Simulation Variation SSP126")

# fill between SAM historical (rolling avg) and ssp585 variation
ax.fill_between(sam_sim_vari.loc[historical_start:historical_end].index, 
                sam_sim_vari.loc[historical_start:historical_end, "SAM Historical (Rolling)"] - sam_sim_vari.loc[historical_start:historical_end, "Simulation Variation SSP585 (Rolling)"], 
                sam_sim_vari.loc[historical_start:historical_end, "SAM Historical (Rolling)"] + sam_sim_vari.loc[historical_start:historical_end, "Simulation Variation SSP585 (Rolling)"], 
                color='silver', alpha=0.3, label="Simulation Variation SSP585")

ax.set_title("")
ax.legend(loc='upper left')
ax.set_ylim(-2, 3.0)
ax.set_yticks([-2, -1, 0, 1, 2, 3])
ax.set_ylabel("SAM Index")
plt.xlabel("Year")

plt.tight_layout()
plt.show()

# Figure 2: Establishing Link between SAM and SST w/ Impact Across SSP Scenarios

## Figure 2a: SST and SAM (with regression correlation) Across All Scenarios - Historical Timeseries

In [ ]:
# hist
sst_reg1 = 'sam_sst_regression_ann'
samsst_hist = hist[sst_reg1]
samsst_hist

In [ ]:
# ssp126
sst_reg2 = 'sam_sst_regression_ann'
samsst_ssp126 = ssp126hist[sst_reg2]
samsst_ssp126

In [ ]:
# ssp585
sst_reg3 = 'sam_sst_regression_ann'
samsst_ssp585 = ssp585hist[sst_reg3]
samsst_ssp585

In [ ]:
# lat lon for plotting
lat= hist['lat']
lon= hist['lon']

In [ ]:
samsst_hist.plot()

In [ ]:
cmap = plt.get_cmap('PuOr')
cmap_limits = [-2, 2]
levels = np.linspace(cmap_limits[0], cmap_limits[1], 51)
norm = mcolors.BoundaryNorm(boundaries=levels, ncolors=256)

projection = ccrs.Orthographic(central_latitude=-90, central_longitude=0)
transform = ccrs.PlateCarree()

scenario = ['Historical', 'SSP126', 'SSP585']
datasets = [samsst_hist, samsst_ssp126, samsst_ssp585]

# subplots
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14, 10), subplot_kw={'projection': projection}, constrained_layout=True)

tick_values = np.linspace(cmap_limits[0], cmap_limits[1], 5)

for data, ax, scenario in zip(datasets, axs.flat, scenario):
    
    samsst_hist = data
    # contours for sst
    im = ax.contourf(lon, lat, samsst_hist, levels=levels, transform=transform, cmap=cmap, norm=norm)
    # plot features
    ax.set_title(f'SAM {scenario}')
    ax.coastlines()
    gl = ax.gridlines(draw_labels=True)

cbar = fig.colorbar(im, ax=axs, orientation='horizontal', pad=0.05, shrink=0.8, aspect= 40, ticks=tick_values)
cbar.set_label('Temperature (°C)')

plt.show()

## Figure 2b: Plotting SST Across All Scenarios

In [ ]:
# hist
sic_var1 = 'sic_sh_spatialmean_ann'
sic_sh_hist1 = hist[sic_var1]
sic_sh_hist1

In [ ]:
sic_sh_hist = sic_sh_hist1.where(sic_sh_hist1>0)
sic_sh_hist.plot()

In [ ]:
# ssp126
sic_var2 = 'sic_sh_spatialmean_ann'
sic_sh_ssp1261 = ssp126mean[sic_var2]
sic_sh_ssp1261

In [ ]:
sic_sh_ssp126 = sic_sh_ssp1261.where(sic_sh_ssp1261>0)
sic_sh_ssp126.plot()

In [ ]:
# ssp585
sic_var3 = 'sic_sh_spatialmean_ann'
sic_sh_ssp5851 = ssp585mean[sic_var3]
sic_sh_ssp5851

In [ ]:
sic_sh_ssp585 = sic_sh_ssp5851.where(sic_sh_ssp5851>0)
sic_sh_ssp585.plot()

In [ ]:
# lat lon for ice grid
lon2d=hist['lon2d_ice_sh']
lat2d=hist['lat2d_ice_sh']

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(14,4))
lat2d.plot(ax=ax1)
lon2d.plot(ax=ax2)

In [ ]:
variables = [sic_sh_hist, sic_sh_ssp126, sic_sh_ssp585]
scenarios = ['Historical', 'SSP126', 'SSP585']

cmap = plt.get_cmap('Blues')
projection = ccrs.Orthographic(central_latitude=-90, central_longitude=0)
transform = ccrs.PlateCarree()

bounds = np.linspace(0, 100, 6)
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=256)

# subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 6), subplot_kw={'projection': projection}, 
                        gridspec_kw={'wspace': 0.1})

for ax, var, scenario in zip(axs, variables, scenarios):
    ax.add_feature(cfeature.LAND, zorder=1, facecolor=cfeature.COLORS['land_alt1'])
    ax.coastlines(resolution='110m', linewidth=0.5)
    ax.gridlines()

    pcm = ax.pcolormesh(lon2d, lat2d, var, transform=transform, cmap=cmap, norm=norm)

    ax.set_title(scenario)

cbar_height = 0.03
cbar_width = 0.6  
cbar_ax = fig.add_axes([0.2, 0.06, cbar_width, cbar_height])  
cbar = plt.colorbar(pcm, cax=cbar_ax, orientation='horizontal')
cbar.set_label('Antarctic Sea Ice Concentration (%)')

plt.show()

In [ ]:
sst_vars = [var for var in hist.variables if 'tas' in var]
sst_vars

In [ ]:
# hist
tas_var1 = 'tas_spatialmean_ann'
tas_hist_spat = hist[tas_var1]
tas_hist_spat.plot()

In [ ]:
# ssp126
tas_var2 = 'tas_spatialmean_ann'
tas_126_spat = ssp126mean[tas_var2]
tas_126_spat.plot()

In [ ]:
# ssp585
tas_var3 = 'tas_spatialmean_ann'
tas_585_spat = ssp585mean[tas_var3]
tas_585_spat.plot()

sst_var2 = 'sst_spatialmean_ann'
sst_hist_spat = hist[sst_var2]
sst_hist_spat

sst_var3 = 'sst_spatialmean_ann'
sst_126_spat = ssp126mean[sst_var3]
sst_126_spat.plot()

sst_var4 = 'sst_spatialmean_ann'
sst_585_spat = ssp585mean[sst_var4]
sst_585_spat.plot()

In [ ]:
lon = hist['longitude']
lat = hist['latitude']

In [ ]:
cmap = plt.get_cmap('coolwarm')
cmap_limits = [-80, 50]
levels = mcolors.BoundaryNorm(boundaries=range(-80, 51, 5), ncolors=256)

projection = ccrs.Orthographic(central_latitude=-90, central_longitude=0)
transform = ccrs.PlateCarree()

seasons = ['Historical', 'SSP126', 'SSP585']
datasets = [tas_hist_spat, tas_126_spat, tas_585_spat]

# subplots
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14, 10), subplot_kw={'projection': projection}, constrained_layout=True)

for data, ax, season in zip(datasets, axs.flat, seasons):
    lat = hist['lat']
    lon = hist['lon']
    
    tas_hist_spat = data
    # contours
    im = ax.contourf(lon, lat, tas_hist_spat, transform=transform, cmap=cmap, norm=levels)
    
    ax.set_title(f'SAM {season}')
    ax.coastlines()
    gl = ax.gridlines(draw_labels=True)
    
cbar = fig.colorbar(im, ax=axs, orientation='horizontal', pad=0.05, shrink=0.8, aspect=40)
cbar.set_label('Annual Sea Surface Temperature (C)')
cbar.set_ticks(range(-60, 50, 10)) 

plt.show()

## Figure 2c: Plotting TAS and SIC Across All Scenarios

In [ ]:
sic_variables = [sic_sh_hist, sic_sh_ssp126, sic_sh_ssp585]
tas_variables = [tas_hist_spat, tas_126_spat, tas_585_spat]
scenarios = ['Historical', 'SSP126', 'SSP585']

fig, axs = plt.subplots(1, 3, figsize=(20, 10), subplot_kw={'projection': ccrs.Orthographic(central_latitude=-90, central_longitude=0)}, gridspec_kw={'wspace': 0.1})

# sic and tas
for ax, sic_var, tas_var, scenario in zip(axs, sic_variables, tas_variables, scenarios):
    
    cmap_sic = plt.get_cmap('Blues')
    bounds_sic = np.linspace(0, 100, 6)
    levels_sic = np.linspace(0, 100, 6)
    cmap_tas = plt.get_cmap('bwr')
    pcm_sic = ax.pcolormesh(lon2d, lat2d, sic_var, transform=ccrs.PlateCarree(), cmap=cmap_sic, vmin=0, vmax=100)

    # tas contours
    cf = ax.contour(lon, lat, tas_var, levels=range(-55, 35, 5), cmap=cmap_tas, linewidths=1.2, transform=ccrs.PlateCarree(), alpha=0.7)
    # tas contours labels
    ax.clabel(cf, fmt='%d', inline=True, fontsize=8)

    ax.set_title(scenario)

    ax.add_feature(cfeature.LAND, zorder=1, facecolor='lightgray')
    ax.coastlines(resolution='110m', linewidth=0.5)
    ax.gridlines(draw_labels=True)

cbar_sic = fig.colorbar(pcm_sic, ax=axs, orientation='horizontal', pad=0.05, shrink=0.75, aspect=50)
cbar_sic.set_label('Antarctic Sea Ice Concentration (%)')

plt.show()

## Figure 3a: Plotting Mean SAM and SIC Across All Scenarios

In [ ]:
sam_vars = [var for var in hist.variables if 'sam' in var]
sam_vars

In [ ]:
# hist
sam_var1 = 'sam_ann'
sam_hist_ann1 = hist[sam_var1]
sam_hist_ann1

In [ ]:
# ssp126
sam_var2 = 'sam_ann'
sam_ssp126_ann1 = ssp126mean[sam_var2]
sam_ssp126_ann1.plot()

In [ ]:
# ssp585
sam_var3 = 'sam_ann'
sam_ssp585_ann1 = ssp585mean[sam_var3]
sam_ssp585_ann1.plot()

In [ ]:
# SIC Variables: sic_sh_hist, sic_sh_ssp126, sic_sh_ssp585

In [ ]:
sam_hist_ann1

In [ ]:
sic_data = [sic_sh_hist, sic_sh_ssp126, sic_sh_ssp585]
sic_seasons = ['Historical', 'SSP126', 'SSP585']
sam_data = [sam_hist_ann1, sam_ssp126_ann1, sam_ssp585_ann1]
sam_seasons = ['Historical', 'SSP126', 'SSP585']

cmap_sic = plt.get_cmap('Blues')
cmap_sam = plt.get_cmap('PRGn')
cmap_limits_sam = [-5, 5]
bounds_sic = np.linspace(0, 100, 6)

# contour levels for SAM 0.5 hPa
min_sam = np.floor(np.min([np.min(arr) for arr in sam_data]))
max_sam = np.ceil(np.max([np.max(arr) for arr in sam_data]))
levels_sam = np.arange(min_sam, max_sam + 0.5, 0.5)

norm_sic = mcolors.BoundaryNorm(boundaries=bounds_sic, ncolors=256)
norm_sam = mcolors.BoundaryNorm(boundaries=levels_sam, ncolors=256)
projection = ccrs.Orthographic(central_latitude=-90, central_longitude=0)
transform = ccrs.PlateCarree()

# subplots
fig, axs = plt.subplots(1, 3, figsize=(14, 10), subplot_kw={'projection': projection}, constrained_layout=True)

for col in range(3):
    # SIC
    ax = axs[col]
    ax.add_feature(cfeature.LAND, zorder=1, facecolor=cfeature.COLORS['land_alt1'])
    ax.coastlines(resolution='110m', linewidth=0.5)
    ax.gridlines()
    
    pcm_sic = ax.pcolormesh(lon2d, lat2d, sic_data[col], transform=transform, cmap=cmap_sic, norm=norm_sic)
    ax.set_title(f'SIC {sic_seasons[col]}')
    # setting specific region
    ax.set_extent([lon.min() + 60, lon.max() - 30, lat.min() + 60, lat.max() - 35], crs=ccrs.PlateCarree())

    # SAM contours with labels
    cs_sam = ax.contour(lon, lat, sam_data[col], levels=levels_sam, transform=transform,
                        cmap=cmap_sam, norm=norm_sam, alpha=1)
    ax.clabel(cs_sam, inline=True, fontsize=8)

cb_height = 0.03
cb_pad = 1 

cb_ax_sic = fig.add_axes([0.1, 0.2, 0.8, cb_height])
cbar_sic = plt.colorbar(pcm_sic, cax=cb_ax_sic, orientation='horizontal', ticks=np.arange(0, 101, 10))
cbar_sic.ax.tick_params(labelsize=8) 
cbar_sic.set_label('Antarctic Sea Ice Concentration (%)')

plt.show()

## Figure 3b: Plotting Mean SAM and SIC Across All Seasonal Scenarios 

### Seasonal SIC for All Scenarios:

In [ ]:
# hist winter
sic_var4 = 'sic_sh_spatialmean_jja'
sic_sh_jja1 = hist[sic_var4]
sic_hist_winter = sic_sh_jja1.where(sic_sh_jja1>0)
sic_hist_winter

In [ ]:
# hist summer
sic_var5 = 'sic_sh_spatialmean_djf'
sic_sh_djf1 = hist[sic_var5]
sic_hist_summer = sic_sh_djf1.where(sic_sh_djf1>0)
sic_hist_summer

In [ ]:
# ssp126 winter
sic_var6 = 'sic_sh_spatialmean_jja'
sic_sh_jja2 = ssp126mean[sic_var6]
sic_ssp126_winter = sic_sh_jja2.where(sic_sh_jja2>0)
sic_ssp126_winter

In [ ]:
# ssp126 summer
sic_var7 = 'sic_sh_spatialmean_djf'
sic_sh_djf2 = ssp126mean[sic_var7]
sic_ssp126_summer = sic_sh_djf2.where(sic_sh_djf2>0)
sic_ssp126_summer

In [ ]:
# ssp585 winter
sic_var8 = 'sic_sh_spatialmean_jja'
sic_sh_jja3 = ssp585mean[sic_var8]
sic_ssp585_winter = sic_sh_jja3.where(sic_sh_jja3>0)
sic_ssp585_winter

In [ ]:
# ssp585 summer
sic_var9 = 'sic_sh_spatialmean_djf'
sic_sh_djf3 = ssp585mean[sic_var9]
sic_ssp585_summer = sic_sh_djf3.where(sic_sh_djf3>0)
sic_ssp585_summer

### Seasonal SAM for All Scenarios:
sam_djf',
 'sam_jja

In [ ]:
# hist winter
sam_var4 = 'sam_jja'
sam_hist_winter = hist[sam_var4]
sam_hist_winter

In [ ]:
# hist summer
sam_var5 = 'sam_djf'
sam_hist_summer = hist[sam_var5]
sam_hist_summer

In [ ]:
# ssp126 winter
sam_var6 = 'sam_jja'
sam_ssp126_winter = ssp126mean[sam_var6]
sam_ssp126_winter

In [ ]:
# ssp126 summer
sam_var7 = 'sam_djf'
sam_ssp126_summer = ssp126mean[sam_var7]
sam_ssp126_summer

In [ ]:
# ssp585 summer
sam_var8 = 'sam_djf'
sam_ssp585_summer = ssp585mean[sam_var8]
sam_ssp585_summer

In [ ]:
# ssp585 winter
sam_var9 = 'sam_jja'
sam_ssp585_winter = ssp585mean[sam_var9]
sam_ssp585_winter

In [ ]:
# lat lon for ice grid
lon2d=hist['lon2d_ice_sh']
lat2d=hist['lat2d_ice_sh']

In [ ]:
lat= hist['lat']
lon= hist['lon']

In [ ]:
#sic_data = [sic_sh_hist, sic_sh_ssp126, sic_sh_ssp585]
#sic_seasons = ['Historical', 'SSP126', 'SSP585']
#sam_data = [sam_hist_ann1, sam_ssp126_ann1, sam_ssp585_ann1]
#sam_seasons = ['Historical', 'SSP126', 'SSP585']

sic_data = [sic_sh_hist, sic_sh_ssp126, sic_sh_ssp585, sic_hist_winter, sic_ssp126_winter, sic_ssp585_winter, sic_hist_summer, sic_ssp126_summer, sic_ssp585_summer]
sic_seasons = ['Historical Annual', 'SSP126 Annual', 'SSP585 Annual', 'Historical Winter', 'SSP126 Winter', 'SSP585 Winter', 'Historical Summer', 'SSP126 Summer', 'SSP585 Summer']

sam_data = [sam_hist_ann1, sam_ssp126_ann1, sam_ssp585_ann1, sam_hist_winter, sam_ssp126_winter, sam_ssp585_winter, sam_hist_summer, sam_ssp126_summer, sam_ssp585_summer]
sam_seasons = ['Historical Annual', 'SSP126 Annual', 'SSP585Annual', 'Historical Winter', 'SSP126 Winter', 'SSP585 Winter', 'Historical Summer', 'SSP126 Summer', 'SSP585 Summer']

cmap_sic = plt.get_cmap('Blues')
cmap_sam = plt.get_cmap('PRGn')
bounds_sic = np.linspace(0, 100, 6)

# contours for SAM 0.5 hPa intervals
min_sam = np.floor(np.min(sam_data))
max_sam = np.ceil(np.max(sam_data))
levels_sam = np.arange(min_sam, max_sam + 0.5, 0.5)

norm_sic = mcolors.BoundaryNorm(boundaries=bounds_sic, ncolors=256)
norm_sam = mcolors.BoundaryNorm(boundaries=levels_sam, ncolors=256)

# subplots
fig, axs = plt.subplots(3, 3, figsize=(20, 15), subplot_kw={'projection': ccrs.Orthographic(central_latitude=-90, central_longitude=0)}, constrained_layout=True, gridspec_kw={'wspace': 0.01})

for idx, (sic_var, sam_var, sic_season, sam_season) in enumerate(zip(sic_data, sam_data, sic_seasons, sam_seasons)):
    row_idx = idx // 3  # row index
    col_idx = idx % 3  # column index

    ax = axs[row_idx, col_idx]

    # SIC
    pcm_sic = ax.pcolormesh(lon2d, lat2d, sic_var, transform=ccrs.PlateCarree(), cmap=cmap_sic, norm=norm_sic)
    ax.set_title(f'SIC {sic_season}')
    ax.coastlines(resolution='110m', linewidth=0.5)
    ax.gridlines()
    ax.set_extent([lon.min() + 60, lon.max() - 30, lat.min() + 60, lat.max() - 35], crs=ccrs.PlateCarree())

    # SAM with contours
    cs_sam = ax.contour(lon, lat, sam_var, levels=levels_sam, transform=ccrs.PlateCarree(), cmap=cmap_sam, norm=norm_sam, alpha = 0.6)
    ax.clabel(cs_sam, inline=True, fontsize=8)  # contour labels

cb_width = 0.01 
cb_pad = 0.01

cb_ax_sic = fig.add_axes([0.01, 0.01, cb_width, 1]) 
cbar_sic = plt.colorbar(pcm_sic, cax=cb_ax_sic)
cbar_sic.ax.tick_params(labelsize=8)
cbar_sic.set_label('Antarctic Sea Ice Concentration (%)')

plt.show()

## Figure 3c: Multiple timeseries Showing Monthly SAM and SIC With Different Time Scales

In [ ]:
# sam hist timeseries
sam_var10 = 'sam_timeseries_mon'
sam_hist_trend = hist[sam_var10]
sam_hist_trend

In [ ]:
# sam ssp126 timeseries
sam_var11 = 'sam_timeseries_mon'
sam_ssp126_trend = ssp126hist[sam_var11]
sam_ssp126_trend

In [ ]:
# sam ssp585 timeseries
sam_var12 = 'sam_timeseries_mon'
sam_ssp585_trend = ssp585hist[sam_var12]
sam_ssp585_trend

In [ ]:
# switching to df to understand data and make averaging easier
sam_hist_df = sam_hist_trend.to_dataframe(name='SAM_Hist')

print(sam_hist_df)

In [ ]:
datetime_index = [date.strftime('%Y-%m-%d') for date in sam_hist_df.index]

# indexing
sam_hist_df.index = pd.to_datetime(datetime_index)

# setting time period for 2004-2014
start_date = '1981-01-01'
end_date = '2010-12-31'
sam_hist_subset = sam_hist_df.loc[(sam_hist_df.index >= start_date) & (sam_hist_df.index <= end_date)]

print(sam_hist_subset)

In [ ]:
# changing to month
sam_hist_subset['Month'] = sam_hist_subset.index.strftime('%B')

# grouping by month, calc monthly avgs
monthly_avg = sam_hist_subset.groupby('Month')['SAM_Hist'].mean().reset_index()

print(monthly_avg)

In [ ]:
# setting to 8d.p.
pd.options.display.float_format = '{:.8f}'.format

monthly_avg.head(12)

In [ ]:
sam_126_df = sam_ssp126_trend.to_dataframe(name='SAM_126')

print(sam_126_df)

In [ ]:
datetime_index2 = [date.strftime('%Y-%m-%d') for date in sam_126_df.index]

# indexing
sam_126_df.index = pd.to_datetime(datetime_index2)

# setting time period for 2071-2100
start_date = '2071-01-01'
end_date = '2100-12-31'
sam_126_subset = sam_126_df.loc[(sam_126_df.index >= start_date) & (sam_126_df.index <= end_date)]

sam_126_subset.head(13)

In [ ]:
mean_sam_126 = sam_126_subset['SAM_126'].mean()
std_sam_126 = sam_126_subset['SAM_126'].std()

print("Mean average of SAM_126:", mean_sam_126)
print("Standard deviation of SAM_126:", std_sam_126)


In [ ]:
sam_585_df = sam_ssp585_trend.to_dataframe(name='SAM_585')

sam_585_df.head()

In [ ]:
datetime_index3 = [date.strftime('%Y-%m-%d') for date in sam_585_df.index]

# indexing
sam_585_df.index = pd.to_datetime(datetime_index3)

# setting time period 2071-2100
start_date = '2071-01-01'
end_date = '2100-12-31'
sam_585_subset = sam_585_df.loc[(sam_585_df.index >= start_date) & (sam_585_df.index <= end_date)]

sam_585_subset.head()

In [ ]:
mean_sam_585 = sam_585_subset['SAM_585'].mean()
std_sam_585 = sam_585_subset['SAM_585'].std()

print("Mean average of SAM_585:", mean_sam_585)
print("Standard deviation of SAM_585", std_sam_585)

In [ ]:
sam_585_subset['Month'] = sam_585_subset.index.month

# grouping by month and calc avgs
monthly_avg_585 = sam_585_subset.groupby('Month')['SAM_PC'].mean().reset_index()

# new df
monthly_avg585_df = pd.DataFrame({
    'Month': monthly_avg['Month'],
    'SAM_Mean585': monthly_avg['SAM_PC']
})

monthly_avg_585['Month'] = pd.to_datetime(monthly_avg_585['Month'], format='%m').dt.strftime('%B')

monthly_avg_585.head(13)

In [ ]:
# merging dfs with month
sam_mon_avgf = pd.merge(monthly_avg, monthly_avg_585, on='Month')
sam_mon_avgf = pd.merge(monthly_avg, monthly_avg_126, on='Month')

sam_mon_avgf = sam_mon_avgf.rename(columns={'SAM_Mon_Mean': 'SAM_Hist', 'SAM_PC_x': 'SAM_585', 'SAM_PC_y': 'SAM_126'})

sam_mon_avgf.head(13)

In [ ]:
# SAM_585 - SAM_Hist each month
sam_mon_avgf['SAM_585_Anom'] = sam_mon_avgf['SAM_585'] - sam_mon_avgf['SAM_Hist']

# SAM_126 - SAM_Hist each month
sam_mon_avgf['SAM_126_Anom'] = sam_mon_avgf['SAM_126'] - sam_mon_avgf['SAM_Hist']

# new df with calcs
SAM_Future_Mon_AvgAnom = pd.DataFrame({
    'Month': sam_mon_avgf['Month'],
    'SAM_585_Anom': sam_mon_avgf['SAM_585_Anom'],
    'SAM_126_Anom': sam_mon_avgf['SAM_126_Anom']
})

print(SAM_Future_Mon_AvgAnom)

In [ ]:
sie_variables = [var for var in hist.variables if 'sic_sh' in var]
sie_variables

In [ ]:
# hist
sie_var1 = 'sic_sh_extent_mon'
sie_hist_mon = hist[sie_var1]
sie_hist_mon

In [ ]:
# ssp126
sie_var2 = 'sic_sh_extent_mon'
sie_126_mon = ssp126hist[sie_var2]
sie_126_mon

In [ ]:
# ssp585
sie_var3 = 'sic_sh_extent_mon'
sie_585_mon = ssp585hist[sie_var3]
sie_585_mon

In [ ]:
sie_hist_df = sie_hist_mon.to_dataframe(name='SIE')

print(sie_hist_df)

In [ ]:
datetime_index4 = [date.strftime('%Y-%m-%d') for date in sie_hist_df.index]

# indexing
sie_hist_df.index = pd.to_datetime(datetime_index4)

# time period 1981-2010
start_date = '1981-01-01'
end_date = '2010-12-31'
sie_hist_subset = sie_hist_df.loc[(sie_hist_df.index >= start_date) & (sie_hist_df.index <= end_date)]

sie_hist_subset.head(13)

In [ ]:
sie_hist_subset['Month'] = sie_hist_subset.index.month

# grouping by month, calc avg SIE
siemonthly_avg_hist = sie_hist_subset.groupby('Month')['SIE'].mean().reset_index()

# new df with SIE values
siemonthly_avg_hist = pd.DataFrame({
    'Month': siemonthly_avg_hist['Month'],
    'SIE_Mean': siemonthly_avg_hist['SIE']
})

siemonthly_avg_hist['Month'] = pd.to_datetime(siemonthly_avg_hist['Month'], format='%m').dt.strftime('%B')

siemonthly_avg_hist.head(13)

In [ ]:
sie_126_df = sie_126_mon.to_dataframe(name='SIE')

print(sie_126_df)

In [ ]:
datetime_index5 = [date.strftime('%Y-%m-%d') for date in sie_126_df.index]

# indexing
sie_126_df.index = pd.to_datetime(datetime_index5)

# time period 2071-2100
start_date = '2071-01-01'
end_date = '2100-12-31'
sie_126_subset = sie_126_df.loc[(sie_126_df.index >= start_date) & (sie_126_df.index <= end_date)]

sie_126_subset.head(13)

In [ ]:
sie_126_subset['Month'] = sie_126_subset.index.month

# grouping by month, calc avg SIE
siemonthly_avg_126 = sie_126_subset.groupby('Month')['SIE'].mean().reset_index()

# new df with month and average SIE
siemonthly_avg_126 = pd.DataFrame({
    'Month': siemonthly_avg_126['Month'],
    'SIE_Mean126': siemonthly_avg_126['SIE']
})

siemonthly_avg_126['Month'] = pd.to_datetime(siemonthly_avg_126['Month'], format='%m').dt.strftime('%B')

siemonthly_avg_126.head(13)

In [ ]:
sie_585_df = sie_585_mon.to_dataframe(name='SIE')

print(sie_585_df)

In [ ]:
datetime_index6 = [date.strftime('%Y-%m-%d') for date in sie_585_df.index]

# indexing
sie_585_df.index = pd.to_datetime(datetime_index6)

# time period 2071-2100
start_date = '2071-01-01'
end_date = '2100-12-31'
sie_585_subset = sie_585_df.loc[(sie_585_df.index >= start_date) & (sie_585_df.index <= end_date)]

sie_585_subset.head(13)

In [ ]:
sie_585_subset['Month'] = sie_585_subset.index.month

# grouping by month, calc avg SIE
siemonthly_avg_585 = sie_585_subset.groupby('Month')['SIE'].mean().reset_index()

# new df with month and average SIE
siemonthly_avg_585 = pd.DataFrame({
    'Month': siemonthly_avg_585['Month'],
    'SIE_Mean585': siemonthly_avg_585['SIE']
})

siemonthly_avg_585['Month'] = pd.to_datetime(siemonthly_avg_585['Month'], format='%m').dt.strftime('%B')

siemonthly_avg_585.head(13)

In [ ]:
# merging dfs across month
sie_mon_avgf = pd.merge(siemonthly_avg_hist, siemonthly_avg_585, on='Month')
sie_mon_avgf = pd.merge(sie_mon_avgf, siemonthly_avg_126, on='Month')

# renaming columns
sie_mon_avgf = sie_mon_avgf.rename(columns={'SIE_Mon_Mean': 'SIE_Hist', 'SIE_': 'SAM_585', 'SAM_PC_y': 'SAM_126'})

sie_mon_avgf.head(13)

In [ ]:
# SIE_585 - SIE_Hist for each month
sie_mon_avgf['SIE_585_Anom'] = sie_mon_avgf['SIE_Mean585'] - sie_mon_avgf['SIE_Mean']

# SIE_126 - SIE_Hist for each month
sie_mon_avgf['SIE_126_Anom'] = sie_mon_avgf['SIE_Mean126'] - sie_mon_avgf['SIE_Mean']

# new df with Month column and calcd avgs
SIE_Future_Mon_AvgAnom = pd.DataFrame({
    'Month': sie_mon_avgf['Month'],
    'SIE_585_Anom': sie_mon_avgf['SIE_585_Anom'],
    'SIE_126_Anom': sie_mon_avgf['SIE_126_Anom']
})

print(SIE_Future_Mon_AvgAnom.head)

In [ ]:
SAM_SIE_Anom_df = pd.merge(SAM_Future_Mon_AvgAnom, SIE_Future_Mon_AvgAnom, on='Month')

print(SAM_SIE_Anom_df)

In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 3))

sie_585_anom_line, = ax1.plot(SAM_SIE_Anom_df['Month'], SAM_SIE_Anom_df['SIE_585_Anom'], color='darkcyan', label='SIE_585_Anom')
ax1.set_ylabel('Sea Ice Extent ($10^{12}$ m$^2$)')

ax2 = ax1.twinx()

sam_585_anom_line, = ax2.plot(SAM_SIE_Anom_df['Month'], SAM_SIE_Anom_df['SAM_585_Anom'], color='indianred', label='SAM_585_Anom')
ax2.set_ylabel('Southern Annular Mode Index')

ax1.set_xlabel('Month')

ax1.set_ylim(-9.0, 4.2)
ax2.set_ylim(-0.2, 1.2)

legend_handles = [sie_585_anom_line, sam_585_anom_line]
legend_labels = ['Sea Ice Extent', 'Southern Annular Mode Index']
ax1.legend(legend_handles, legend_labels, loc='upper left')

ax1.grid(True)

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 3))

sie_126_anom_line, = ax1.plot(SAM_SIE_Anom_df['Month'], SAM_SIE_Anom_df['SIE_126_Anom'], color='darkcyan', label='SIE_126_Anom')
ax1.set_ylabel('Sea Ice Extent ($10^{12}$ m$^2$)')

ax2 = ax1.twinx()

sam_126_anom_line, = ax2.plot(SAM_SIE_Anom_df['Month'], SAM_SIE_Anom_df['SAM_126_Anom'], color='orangered', label='SAM_126_Anom')
ax2.set_ylabel('Southern Annular Mode Index')

ax1.set_xlabel('Month')

ax1.set_ylim(-1.8, -0.8)
ax2.set_ylim(-1.0, 1.2)

legend_handles = [sie_126_anom_line, sam_126_anom_line]
legend_labels = ['Sea Ice Extent', 'Southern Annular Mode Index']
ax1.legend(legend_handles, legend_labels, loc='upper left')

ax1.grid(True)

plt.show()

## Redo: Figure 2a - Plotting TAS and SST Regressed with SAM as timeseries

In [ ]:
sam_vars

In [ ]:
sam_var10 = 'sam_sst_regression_ann'
sam_sst_reg_hist = hist[sam_var10]
sam_sst_reg_hist.plot()

In [ ]:
sie_126_df2 = sie_126_mon.to_dataframe(name='SIE126')

sie_126_df2.head(13)

In [ ]:
sam_126_df2 = sam_ssp126_trend.to_dataframe(name='SAM126')

sam_126_df2.head(13)

In [ ]:
sam_sie_df = pd.merge(sie_126_df2, sam_126_df2, left_index=True, right_index=True)

# time as seperate column
sam_sie_df.reset_index(inplace=True)

print(sam_sie_df)

In [ ]:
# making new df
sam_sie_df['time'] = pd.to_datetime(sam_sie_df['time'].astype(str))

sam_sie_df['Year'] = sam_sie_df['time'].dt.year

# grouping by year with max SIE126 value within each year
max_sie_idx = sam_sie_df.groupby('Year')['SIE126'].idxmax()

# new df with max SIE126 and corresponding SAM126 value
MaxSIE_SAM_df = sam_sie_df.loc[max_sie_idx, ['Year', 'SIE126', 'SAM126']]

print(MaxSIE_SAM_df)

In [ ]:
MaxSIE_SAM_df['SIE126_rolling'] = MaxSIE_SAM_df['SIE126'].rolling(window=5, min_periods=1).mean()
MaxSIE_SAM_df['SAM126_rolling'] = MaxSIE_SAM_df['SAM126'].rolling(window=5, min_periods=1).mean()

fig, ax1 = plt.subplots(figsize=(16, 5))

# rolling averages
line1, = ax1.plot(MaxSIE_SAM_df['Year'], MaxSIE_SAM_df['SIE126_rolling'], color='darkcyan', label='Maximum Sea Ice Extent', alpha=1)

# shaded areas as raw data
ax1.fill_between(MaxSIE_SAM_df['Year'], MaxSIE_SAM_df['SIE126'], MaxSIE_SAM_df['SIE126_rolling'], color='darkcyan', alpha=0.2)

ax1.set_xlabel('Year', fontsize=14)
ax1.set_ylabel('Sea Ice Extent ($10^{12}$ m$^2$)', color='black', fontsize=14)
ax1.tick_params(axis='y', labelcolor='black')

ax1.set_yticks(range(10, 23, 2))

ax1.set_ylim(10, 22)

ax2 = ax1.twinx()
line2, = ax2.plot(MaxSIE_SAM_df['Year'], MaxSIE_SAM_df['SAM126_rolling'], color='blue', label='Southern Annular Mode', alpha=0.3)

ax2.fill_between(MaxSIE_SAM_df['Year'], MaxSIE_SAM_df['SAM126'], MaxSIE_SAM_df['SAM126_rolling'], color='blue', alpha=0.05)

ax2.set_ylabel('Southern Annular Mode Index', color='black', fontsize=14)
ax2.tick_params(axis='y', labelcolor='black')

ax2.set_ylim(-3.2, 2)

ax1.grid(axis='y', color='gray', linestyle='--')
ax1.grid(axis='x', color='gray', linestyle='--')

lines = [line1, line2]
labels = [line.get_label() for line in lines]

ax1.legend(lines, labels, loc='lower left', fontsize=12)

fig.tight_layout()
plt.show()

In [ ]:
sie_585_df3 = sie_585_mon.to_dataframe(name='SIE585')

print(sie_585_df3.head)

In [ ]:
sam_585_df3 = sam_ssp585_trend.to_dataframe(name='SAM585')

print(sam_585_df3.head)

In [ ]:
sam_sie_df3 = pd.merge(sie_585_df3, sam_585_df3, left_index=True, right_index=True)

# indexing
sam_sie_df3.reset_index(inplace=True)

print(sam_sie_df3)

In [ ]:
# new df
sam_sie_df3['time'] = pd.to_datetime(sam_sie_df3['time'].astype(str))

sam_sie_df3['Year'] = sam_sie_df3['time'].dt.year

# grouping by year, finding max SIE126 value within each year
max_sie_idx2 = sam_sie_df3.groupby('Year')['SIE585'].idxmax()

# new df with max SIE126 and corresponding SAM126 value
MaxSIE_SAM_df2 = sam_sie_df3.loc[max_sie_idx2, ['Year', 'SIE585', 'SAM585']]

print(MaxSIE_SAM_df2)

In [ ]:
# plotting
MaxSIE_SAM_df2['SIE585_rolling'] = MaxSIE_SAM_df2['SIE585'].rolling(window=5, min_periods=1).mean()
MaxSIE_SAM_df2['SAM585_rolling'] = MaxSIE_SAM_df2['SAM585'].rolling(window=5, min_periods=1).mean()

fig, ax1 = plt.subplots(figsize=(16, 5))

line1, = ax1.plot(MaxSIE_SAM_df2['Year'], MaxSIE_SAM_df2['SIE585_rolling'], color='darkcyan', label='Maximum Sea Ice Extent', alpha=1)

ax1.fill_between(MaxSIE_SAM_df2['Year'], MaxSIE_SAM_df2['SIE585'], MaxSIE_SAM_df2['SIE585_rolling'], color='darkcyan', alpha=0.2)

ax1.set_xlabel('Year', fontsize=14)
ax1.set_ylabel('Sea Ice Extent ($10^{12}$ m$^2$)', color='black', fontsize=14)
ax1.tick_params(axis='y', labelcolor='black')

ax1.set_ylim(6, 22)

ax2 = ax1.twinx()
line2, = ax2.plot(MaxSIE_SAM_df2['Year'], MaxSIE_SAM_df2['SAM585_rolling'], color='blue', label='Southern Annular Mode', alpha=0.3)

ax2.fill_between(MaxSIE_SAM_df2['Year'], MaxSIE_SAM_df2['SAM585'], MaxSIE_SAM_df2['SAM585_rolling'], color='blue', alpha=0.05)

ax2.set_ylabel('Southern Annular Mode Index', color='black', fontsize=14)
ax2.tick_params(axis='y', labelcolor='black')

ax2.set_ylim(-3.2, 2)

ax1.grid(axis='y')
ax1.grid(axis='x')

lines = [line1, line2]
labels = [line.get_label() for line in lines]

ax1.legend(lines, labels, loc='lower left', fontsize=12)

fig.tight_layout()
plt.show()

In [ ]:
va_file = 'https://esgf.ceda.ac.uk/thredds/dodsC/esg_cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/historical/r1i1p1f2/Amon/va/gn/v20190406/va_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_195001-201412.nc.dods'
ua_file='https://esgf.ceda.ac.uk/thredds/dodsC/esg_cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/historical/r1i1p1f2/Amon/ua/gn/v20190406/ua_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_195001-201412.nc.dods'

In [ ]:
d1=xr.open_dataset(va_file)
va=d1.va
d2=xr.open_dataset(ua_file)
ua=d2.ua
lat=d2.lat
lon=d2.lon-180 #change [0,360] to [-180,180]
plev=d2.plev #pressure in Pa, from surface to top